In [6]:
from transformers import BertTokenizer, TFBertModel,BertForQuestionAnswering,BertForMaskedLM,BertModel
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
import logging
import tensorflow as tf
import torch

# Set the logging level to WARNING (or ERROR) to suppress the warning
logging.getLogger("transformers.modeling_utils").setLevel(logging.WARNING)

In [2]:
tokenizer = BertTokenizer.from_pretrained('./pretrained')
QA = BertForQuestionAnswering.from_pretrained("./QA_model")
typo_fix = T5ForConditionalGeneration.from_pretrained("./typo")
typo_tokenizer = T5Tokenizer.from_pretrained("./typo_tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
tokens = tokenizer("one two three four",return_tensors="pt")

In [8]:
model = BertModel.from_pretrained("./model")

In [23]:
with torch.no_grad():
    outputs = model(**tokens)
vector_representation = outputs.last_hidden_state[:, 0, :]
decoded_tokens = tokenizer.convert_ids_to_tokens(vector_representation.argmax(dim=1))


['[unused564]']

In [ ]:
tokens

In [22]:
tokenizer.convert_ids_to_tokens(end["pooler_output"])

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
sentence = "Ths is an exmple sentence with typos"
input_ids = tokenizer.encode(sentence, return_tensors='pt')
mask_idx = input_ids[0].tolist().index(tokenizer.mask_token_id)
input_ids[0, mask_idx] = tokenizer.mask_token_id
outputs = typo_fix(input_ids)
predictions = outputs[0][0, mask_idx].topk(5)
input_ids[0, mask_idx] = predictions.indices[0].item()
fixed_sent = tokenizer.decode(input_ids[0])
print(fixed_sent)

In [19]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

def typo_correction(input_sentence):

    input_sequence = f"fix typo: {input_sentence}"
    input_ids = typo_tokenizer(input_sequence, return_tensors="pt").input_ids

    # Generate the corrected sequence
    generated_ids = typo_fix.generate(input_ids)

    # Decode the generated sequence
    corrected_sentence = typo_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return corrected_sentence

sentence = "Ths is an exmple sentence with typos"
input_ids = typo_tokenizer(sentence,return_tensors = "pt")["input_ids"]
generated_ids = typo_fix.generate(input_ids)
gen_ids

# fixed_sentence = typo_correction(sentence)
# print("Original sentence:", sentence)
# print("Corrected sentence:", fixed_sentence)

NameError: name 'gen_ids' is not defined

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

def typo_correction(input_sentence, mask_idx, model):
    tokenizer = BertTokenizer.from_pretrained('./pretrained')
    
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt')
    input_ids[0, mask_idx] = tokenizer.mask_token_id
    
    outputs = model(input_ids)
    predictions = outputs[0][0, mask_idx].topk(5)
    input_ids[0, mask_idx] = predictions.indices[0].item()
    
    fixed_sent = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return fixed_sent

# Example usage
sentence = "Ths is an exmple sentence with typos"
mask_idx = 6  # Adjust based on the position of the typo
fixed_sentence = typo_correction(sentence, mask_idx, typo_fix)
print("Original sentence:", sentence)
print("Corrected sentence:", fixed_sentence)

In [15]:
question = "What is panda"
answer ="""The study abroad program (https://linktr.ee/binusstudyabroad) is a program that allows you to go to partner institutions with a credit transfer arrangement. This program can be an alternative enrichment track for those who need to do a 3+1 or (2+1)+1 enrichment semester.

The benefits of taking the program are limitless! It is a rewarding experience that allows you to step out of your comfort zone, immerse yourself in an international environment, experience new cultures, potentially pick up a new language, and see the world from a different perspective. You will create a network for a lifetime and have the key to open the door of global opportunities.

There are several types of programs that you can consider as your path of international study experience. Please look at the comparison table below for the guidelines.

Program Type

Study Abroad	Study Abroad
Fees-Paying

Double Degree

Major	All majors, all campuses	All majors, all campuses	Only for students majoring in:
Computer Science (BINUS @ Greater Jakarta)
Marketing Communication (BINUS @ Greater Jakarta)
Mass Communication (BINUS @ Greater Jakarta)
BINUS @ Senayan
Duration	1 – 2 semesters	1 – 2 semesters	2 – 4 semesters
Credit Transfer	Yes	Yes	Yes
Degree	BINUS Degree	BINUS Degree	BINUS Degree + Host Degree
Semester Departure	Min. 3rd semester Max. 7th semester	Min. 3rd semester Max. 7th semester	Min. 5th semester;
it depends on the Double Degree arrangement
Quota	Limited with competition	Limited with direct placement	Unlimited if it meets the host institution’s requirements
FINANCIAL PART

Cost Component

Study Abroad	Study Abroad
Fees-Paying

Double Degree

Tuition Fee Payment	BINUS Tuition: Yes
Host Tuition: Free	
Non-Senayan Student
BINUS Tuition: Yes
Host Tuition: Yes
Senayan Student
BINUS Tuition: (BP3 or fixed tuition only)
Host Tuition: Yes
BINUS Tuition: Yes
(BP3 or fixed tuition only)
Host Tuition: Yes
Airfare, Visa, Insurance, Living Cost	Self-funded
GENERAL REQUIREMENTS

Semester Departure	Study Abroad	Study Abroad
Fees-Paying

Double Degree
Academic	GPA min 2.75	
Passed required BINUS courses
GPA depends on the host institution’s requirement
Language Proficiency	IELTS Academic 6.0 or​
TOEFL iBT 80 or​
TOEFL ITP 550​

"""
inputs = tokenizer(question, answer, return_tensors="pt")

outputs = QA(**inputs)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)

answer_tokens = inputs["input_ids"][0][start_index:end_index + 1]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
answer

''

In [16]:
start_index

tensor(72)

In [17]:
end_index

tensor(41)

In [4]:
6,1

{'input_ids': tensor([[ 101, 2040, 2003, 8915, 2684, 1997, 1037, 1029,  102, 1038, 1005, 1055,
         2269, 1005, 1055, 1039, 1998, 2014, 2388, 2003, 1037,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
torch.tensor

<function torch._VariableFunctionsClass.tensor>